To do:
- [ ] fix motion correction part
- how do I save out motion corrected brain and how does this affect later analyses
- [ ] calculate pearson correlations, save out that info into another h5

In [ ]:
# import packages
import glob
import h5py
import pandas as pd

from src import io, moco, roi, ttl, zdF
import os
import numpy as np

import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
# set these each time

exp_date = '251014'
exp_num = '1701'

experiment_path = io.get_experiment(exp_date, exp_num)
print(experiment_path)
# set raw imaging and camera files
# if this fails, make sure you're connected to psych server

csv_path = io.get_file(experiment_path,'raw', '.csv')
green_path = io.get_file(experiment_path,'raw', 'channel_1.nii')
xml_path = io.get_file(experiment_path,'raw', '.xml')

# set raw fictrac data
dat_path = io.get_file(experiment_path,'raw', '.dat')

func_brain = io.load(green_path)

n_slices = func_brain.shape[2]  # z
n_vols = func_brain.shape[-1]  # t
print('x, y, z, t', func_brain.shape)

/home/ptarabishi/repos/ROI_analysis/251014_1701


In [7]:
dim = pd.DataFrame(func_brain.shape, )
type(dim)

NameError: name 'func_brain' is not defined

In [5]:

# moco_brain = moco.motion_correction(func_brain, func_brain)
# need to work on this way more

# to extract ROIs

In [6]:
processed_dir = fr'{experiment_path}/processed/'
if not os.path.isdir(processed_dir):
    os.mkdir(processed_dir)
    print("Processed folder created")
else:
    print("Processed folder already exists")

Processed folder already exists


In [ ]:
processed_path = glob.glob(fr'{processed_dir}*supervoxels.h5')
labels_path = glob.glob(fr'{processed_dir}*labels.h5')

# check for labels.h5 or extract ROIs from moco brain
if labels_path:
    print(fr'cluster labels already saved in {labels_path[0]}')
else:
    path = fr'{processed_dir}/{exp_date}_{exp_num}_labels.h5'
    print('cluster labels h5 file made')
    hf = h5py.File(path, 'w')

    print('generating clusters')
    labels = roi.extract_ROIs(func_brain, n_clusters=200)
    hf.create_dataset('labels', data=labels)
    hf.close()

if processed_path:
    print(f'supervoxel data already saved in {processed_path[0]}')
else:
    # open labels file
    labels_hf = h5py.File(labels_path[0], 'r')
    labels = labels_hf['labels'][...]

    path = fr'{processed_dir}/{exp_date}_{exp_num}_supervoxels.h5'
    print('calculating df/F signal')
    df = zdF.calculate_zscoredF(func_brain, labels, n_clusters=200)

    print('saving acquisition frame rates')
    ttls = ttl.read_csv(csv_path)
    scope_timestamps = ttl.extract_2p_relative_timestamps(ttls)
    camera_timestamps = ttl.extract_camera_relative_timestamps(ttls)
    scope_framerate = ttl.get_frame_rate(pd.Series(scope_timestamps))
    camera_framerate = ttl.get_frame_rate(pd.Series(camera_timestamps))

    print('smoothing fictrac speed')
    fictrac_data = pd.DataFrame(pd.read_csv(dat_path, header=None))
    # for window size of 500 ms
    win_size = int(.5 * camera_framerate)
    inst_speed = np.rad2deg(fictrac_data[18])
    smoothed_speed = savgol_filter(inst_speed, win_size, 3)

    with h5py.File(path,'w') as hf:
        hf.create_dataset('ca_signal', data=df)
        hf.create_dataset('scope_fr', data=scope_framerate)
        hf.create_dataset('camera_fr', data=camera_framerate)
        hf.create_dataset('smoothed_speed', data=smoothed_speed)
        hf.create_dataset('fictrac_time', data = fictrac_data[0])
        hf.create_dataset('brain_dimensions', data = dim)

    hf.close()
    print(f'supervoxel data saved as {processed_path[0]}')

cluster labels already saved in /Volumes/AhmedLab/princess/data/250829_1301/processed/TSeries-08292025-1301_channel_1_labels.h5
calculating df/F signal
saving acquisition frame rates
smoothing fictrac speed
supervoxel data saved as /Volumes/AhmedLab/princess/data/250829_1301/processed/250829_1301_supervoxels.h5
